<a href="https://colab.research.google.com/github/ChungbinKim/-/blob/master/TextRank.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install konlpy
import glob
import pickle
import pandas as pd
import numpy as np
import sys
import os
import re
import datetime 
import networkx
import json
from konlpy.tag import Komoran

     |████████████████████████████████| 19.4MB 63.9MB/s 
     |████████████████████████████████| 92kB 8.4MB/s 
     |████████████████████████████████| 460kB 41.6MB/s 
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [2]:
tagger = Komoran()

### 1. 전처리용 클래스 정의

#### 1. A.  원문 텍스트 클린징 + 모델에 문장 할당용

In [3]:
class RawSentence:
    def __init__(self, textIter):
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s

In [4]:
class RawSentenceReader:
    def __init__(self, filepath):
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a, b: a + b, ch[::2], ch[1::2]):
                if not s: continue
                yield s

#### 1. B. Komoran 형태소 분석기를 활용하여 단어의 품사를 태깅

In [5]:
class RawTagger:
    def __init__(self, textIter, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        if type(textIter) == str: self.textIter = textIter.split('\n')
        else: self.textIter = textIter
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in self.textIter:
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)

In [6]:
class RawTaggerReader:
    def __init__(self, filepath, tagger = None):
        if tagger:
            self.tagger = tagger
        else :
            from konlpy.tag import Komoran
            self.tagger = Komoran()
        self.filepath = filepath
        self.rgxSplitter = re.compile('([.!?:](?:["\']|(?![0-9])))')
 
    def __iter__(self):
        for line in open(self.filepath, encoding='utf-8'):
            ch = self.rgxSplitter.split(line)
            for s in map(lambda a,b:a+b, ch[::2], ch[1::2]):
                if not s: continue
                yield self.tagger.pos(s)

### 2. TextRank 클래스 정의

In [7]:
class TextRank:
    def __init__(self, **kargs):
        self.graph = None
        self.window = kargs.get('window', 5)
        self.coef = kargs.get('coef', 1.0)
        self.threshold = kargs.get('threshold', 0.005)
        self.dictCount = {}
        self.dictBiCount = {}
        self.dictNear = {}
        self.nTotal = 0
        
    def load(self, sentenceIter, wordFilter = None):
        def insertPair(a, b):
            if a > b: a, b = b, a
            elif a == b: return
            self.dictBiCount[a, b] = self.dictBiCount.get((a, b), 0) + 1
 
        def insertNearPair(a, b):
            self.dictNear[a, b] = self.dictNear.get((a, b), 0) + 1
 
        for sent in sentenceIter:
            for i, word in enumerate(sent):
                if wordFilter and not wordFilter(word): continue
                self.dictCount[word] = self.dictCount.get(word, 0) + 1
                self.nTotal += 1
                if i - 1 >= 0 and (not wordFilter or wordFilter(sent[i-1])): insertNearPair(sent[i-1], word)
                if i + 1 < len(sent) and (not wordFilter or wordFilter(sent[i+1])): insertNearPair(word, sent[i+1])
                for j in range(i+1, min(i+self.window+1, len(sent))):
                    if wordFilter and not wordFilter(sent[j]): continue
                    if sent[j] != word: insertPair(word, sent[j])
 
    def loadSents(self, sentenceIter, tokenizer = None):
        import math
        def similarity(a, b):
            n = len(a.intersection(b))
            return n / float(len(a) + len(b) - n) / (math.log(len(a)+1) * math.log(len(b)+1))
 
        if not tokenizer: rgxSplitter = re.compile('[\\s.,:;-?!()"\']+')
        sentSet = []
        for sent in filter(None, sentenceIter):
            if type(sent) == str:
                if tokenizer: s = set(filter(None, tokenizer(sent)))
                else: s = set(filter(None, rgxSplitter.split(sent)))
            else: s = set(sent)
            if len(s) < 2: continue
            self.dictCount[len(self.dictCount)] = sent
            sentSet.append(s)
 
        for i in range(len(self.dictCount)):
            for j in range(i+1, len(self.dictCount)):
                s = similarity(sentSet[i], sentSet[j])
                if s < self.threshold: continue
                self.dictBiCount[i, j] = s
 
    def getPMI(self, a, b):
        import math
        co = self.dictNear.get((a, b), 0)
        if not co: return None
        return math.log(float(co) * self.nTotal / self.dictCount[a] / self.dictCount[b])
 
    def getI(self, a):
        import math
        if a not in self.dictCount: return None
        return math.log(self.nTotal / self.dictCount[a])
 
    def build(self):
        self.graph = networkx.Graph()
        self.graph.add_nodes_from(self.dictCount.keys())
        for (a, b), n in self.dictBiCount.items():
            self.graph.add_edge(a, b, weight=n*self.coef + (1-self.coef))
 
    def rank(self):
        return networkx.pagerank(self.graph, weight='weight')
 
    def extract(self, ratio = 0.1):
        ranks = self.rank()
        cand = sorted(ranks, key=ranks.get, reverse=True)[:int(len(ranks) * ratio)]
        pairness = {}
        startOf = {}
        tuples = {}
        for k in cand:
            tuples[(k,)] = self.getI(k) * ranks[k]
            for l in cand:
                if k == l: continue
                pmi = self.getPMI(k, l)
                if pmi: pairness[k, l] = pmi
 
        for (k, l) in sorted(pairness, key=pairness.get, reverse=True):
            print(k[0], l[0], pairness[k, l])
            if k not in startOf: startOf[k] = (k, l)
 
        for (k, l), v in pairness.items():
            pmis = v
            rs = ranks[k] * ranks[l]
            path = (k, l)
            tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
            last = l
            while last in startOf and len(path) < 7:
                if last in path: break
                pmis += pairness[startOf[last]]
                last = startOf[last][1]
                rs *= ranks[last]
                path += (last,)
                tuples[path] = pmis / (len(path) - 1) * rs ** (1 / len(path)) * len(path)
 
        used = set()
        both = {}
        for k in sorted(tuples, key=tuples.get, reverse=True):
            if used.intersection(set(k)): continue
            both[k] = tuples[k]
            for w in k: used.add(w)
 
        #for k in cand:
        #    if k not in used or True: both[k] = ranks[k] * self.getI(k)
 
        return both
 
    def summarize(self, ratio = 0.333):
        r = self.rank()
        ks = sorted(r, key=r.get, reverse=True)[:int(len(r)*ratio)]
        return ' '.join(map(lambda k:self.dictCount[k], sorted(ks)))

### 3. TextRank 알고리즘을 활용하여 문서를 요약하는 함수 정의

- (참고) 요약 비율은 문서의 20%
- (참고) "휴면"이 포함된 문장은 모두 추출

In [8]:
def summarise_contents(x):
    tr = TextRank()

    stopword = set([('있', 'VV'), ('하', 'VV'), ('되', 'VV') ])
    
  
    #tr.loadSents(RawSentence(x), lambda sent: filter(lambda y:y not in stopword and y[1] in ('NNG', 'NNP', 'VV', 'VA','ab'), tagger.pos(sent)))
    tr.loadSents(RawSentence(x), lambda sent: filter(lambda y:y not in stopword and y[1] in ('NNG', 'NNP', 'VV', 'VA'), tagger.pos(sent)))
    tr.build()
    ranks = tr.rank()
    return tr.summarize(0.2)

In [9]:
def extract_resting(x): #여기서 꼭 들어가야 되는 단어 정하기
    
    out = ''
    for line in x.split('\n'):
        if not line.isspace() and '휴면' in line:
            out += line + ' '
    return out.strip()

In [10]:
def extract_resting2(x): #여기서 들어가면 안되는 단어 정하기
    
    out = ''
    for line in x.split('\n'):
        if not line.isspace() and '준수합니다' not in line and '노력합니다' not in line:
            out += line + ' '
    return out.strip()

### 4. Data load

#### 4. A. Data load _ Colab ver.

#### Colab

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [12]:
file_list = glob.glob('./drive/MyDrive/data/*_raw.txt')
regex_idx = re.compile('[0-9]+')
regex_text = re.compile('[^0-9a-zA-Z가-힣.()\n]+')
dict_ = {}

for file in file_list:
    sentence = ''
    idx = re.findall(regex_idx, file)[0]
    with open(file, 'r') as f:
        for line in f.readlines():
            sentence += line
    dict_[int(idx)] = re.sub(regex_text, ' ', sentence)

#### 4. B. Data load _ Local ver.

#### Local

In [13]:
#file_list = glob.glob('../data/*_raw.txt')
#regex_idx = re.compile('[0-9]+')
#regex_text = re.compile('[^0-9a-zA-Z가-힣.()\n]+')
#dict_ = {}

#for file in file_list:
 #   sentence = ''
  #  idx = re.findall(regex_idx, file)[0]
   # with open(file, 'r') as f:
    #     for line in f.readlines():
     #        sentence += line
    #dict_[int(idx)] = re.sub(regex_text, ' ', sentence)

In [14]:
#glob.glob('../data/*_raw.txt')

In [15]:
data = pd.DataFrame.from_dict(dict_, orient='index', columns=['Text'])

### 5. 문서요약 실행

In [16]:
data['Text2'] = data['Text'].map(lambda x : extract_resting2(x))
data['Text_Rank'] = data['Text2'].map(lambda x: (summarise_contents(x)) if len(x) >1000 else "The sentence is too short to summarize.." )
data['Resting'] = data['Text'].map(lambda x :  extract_resting(x))
data['Summary'] = data['Text_Rank'] +'---------------------------------------------------------------------------'+data['Resting']
#data['Summary'] = data['Resting2']

# df.to_csv('sample.csv', index = False) # 결과를 저장
print("Done ! ")

Done ! 


In [17]:
data['Text'][1] # Dataframe의 index는 sample 번호와 일치 ,원문

' 서비스 이용약관 \n\n제1조 목적\n본 약관(이하 약관 )은 웰(이하 회사)와 회원에 관한 제반사항을 규정함을 목적으로 합니다.\n\n제2조 용어의 정의\n 회원 은 회사에 개인정보를 제공하여 회원등록을 한 자로서 회사가 제공하는 서비스를 계속적으로 이용할 수 있는 자를 말합니다.\n\n제3조 약관의 게시 및 변경\n 회사는 약관의 규제에 관한 법률 전자거래기본법 정보통신망이용촉진등에관한법률 소비자보호법 등 관련법을 위배하지 않는 범위에서 이 약관을 개정할 수 있습니다.\n\n제4조 약관 외 준칙 및 관할법원\n 본 약관에 명시되지 않은 사항은 국내 관계법에 준하여 해석합니다.\n 회원과 회사 사이에 분쟁이 발생할 경우 서울민사지방법원을 관할 법원으로 합니다.\n\n제5조 회원가입 및 자격\n 회사가 정한 양식에 따라 회원정보를 기입한 후 회원가입을 신청함으로써 회원으로 등록됩니다.\n 다음 각 호의 1에 해당하는 경우 회사가 임의적으로 회원가입을 인정하지 않거나 회원자격을 박탈할 수 있습니다.\n1. 다른 사람의 명의를 사용하여 신청하였을 경우\n2. 신청 시 필수 내용을 허위로 기재하여 신청하였을 경우\n\n제6조 개인정보의 취득 및 이용\n 회사는 고객의 개인정보의 취득과 이용 보호 등에 관한 사항을 개인정보보호정책 에 정한 바에 따르며 개인정보보호정책은 홈페이지 하단에 상시적으로 게시합니다.\n 회사는 고객으로부터 각종 개인신상에 관한 정보를 제공받을 수 있으며 제공받은 정보는 회사가 제공하는 서비스를 고객이 효율적으로 이용할 수 있도록 하는데 사용될 수 있습니다.\n\n제7조 휴면아이디 관리\n 회원이 2년 이상 로그인 및 서비스 이용 기록이 없는 경우 해당 회원의 아이디는 휴면아이디가 되어 회원 로그인이 정지되고 회사는 휴면아이디의 회원정보를 다른 아이디와 별도로 1년 동안 관리합니다.\n 회사는 1항에 의한 회원 로그인 이용정지 10일 전 이메일을 통하여 이용정지에 대하여 안내하고 이용정지가 되는 경우 다시 이메일을 통하여 이용정지 사실에 대

In [18]:
data['Text2'][1] #원문에서 stopword가 들어간 문장 전체를 제거한 것

'서비스 이용약관   제1조 목적 본 약관(이하 약관 )은 웰(이하 회사)와 회원에 관한 제반사항을 규정함을 목적으로 합니다.  제2조 용어의 정의  회원 은 회사에 개인정보를 제공하여 회원등록을 한 자로서 회사가 제공하는 서비스를 계속적으로 이용할 수 있는 자를 말합니다.  제3조 약관의 게시 및 변경  회사는 약관의 규제에 관한 법률 전자거래기본법 정보통신망이용촉진등에관한법률 소비자보호법 등 관련법을 위배하지 않는 범위에서 이 약관을 개정할 수 있습니다.  제4조 약관 외 준칙 및 관할법원  본 약관에 명시되지 않은 사항은 국내 관계법에 준하여 해석합니다.  회원과 회사 사이에 분쟁이 발생할 경우 서울민사지방법원을 관할 법원으로 합니다.  제5조 회원가입 및 자격  회사가 정한 양식에 따라 회원정보를 기입한 후 회원가입을 신청함으로써 회원으로 등록됩니다.  다음 각 호의 1에 해당하는 경우 회사가 임의적으로 회원가입을 인정하지 않거나 회원자격을 박탈할 수 있습니다. 1. 다른 사람의 명의를 사용하여 신청하였을 경우 2. 신청 시 필수 내용을 허위로 기재하여 신청하였을 경우  제6조 개인정보의 취득 및 이용  회사는 고객의 개인정보의 취득과 이용 보호 등에 관한 사항을 개인정보보호정책 에 정한 바에 따르며 개인정보보호정책은 홈페이지 하단에 상시적으로 게시합니다.  회사는 고객으로부터 각종 개인신상에 관한 정보를 제공받을 수 있으며 제공받은 정보는 회사가 제공하는 서비스를 고객이 효율적으로 이용할 수 있도록 하는데 사용될 수 있습니다.  제7조 휴면아이디 관리  회원이 2년 이상 로그인 및 서비스 이용 기록이 없는 경우 해당 회원의 아이디는 휴면아이디가 되어 회원 로그인이 정지되고 회사는 휴면아이디의 회원정보를 다른 아이디와 별도로 1년 동안 관리합니다.  회사는 1항에 의한 회원 로그인 이용정지 10일 전 이메일을 통하여 이용정지에 대하여 안내하고 이용정지가 되는 경우 다시 이메일을 통하여 이용정지 사실에 대하여 고지합니다.  회원은 회원 로그인 이용정지 이

In [19]:
data['Summary'][1]#요약문 

'서비스 이용약관   제1조 목적 본 약관(이하 약관 )은 웰(이하 회사)와 회원에 관한 제반사항을 규정함을 목적으로 합니다.   회원과 회사 사이에 분쟁이 발생할 경우 서울민사지방법원을 관할 법원으로 합니다.   다음 각 호의 1에 해당하는 경우 회사가 임의적으로 회원가입을 인정하지 않거나 회원자격을 박탈할 수 있습니다.   회사는 1항에 의한 회원 로그인 이용정지 10일 전 이메일을 통하여 이용정지에 대하여 안내하고 이용정지가 되는 경우 다시 이메일을 통하여 이용정지 사실에 대하여 고지합니다.   제8조 회원 탈퇴  회원은 회사에 언제든지 탈퇴를 요청할 수 있으며 지체없이 회원탈퇴를 처리합니다.   제9조 회원에 대한 통지  회사가 회원에 대한 통지를 하는 경우 회원이 회사에 제출한 전자우편주소로 할 수 있습니다.---------------------------------------------------------------------------제7조 휴면아이디 관리  회원이 2년 이상 로그인 및 서비스 이용 기록이 없는 경우 해당 회원의 아이디는 휴면아이디가 되어 회원 로그인이 정지되고 회사는 휴면아이디의 회원정보를 다른 아이디와 별도로 1년 동안 관리합니다.  회원은 회원 로그인 이용정지 이후에 사이트 상에서 아이디와 비밀번호를 입력하고 로그인을 하면 휴면상태 해지신청이 되어 즉시 다시 정상적으로 서비스를 이용할 수 있습니다.  회원이 휴면아이디가 된 후 1년 이내에 휴면상태 해지신청을 하지 않은 경우 해당 회원의 회원정보를 삭제합니다.'

In [20]:
#a=data['Summary'][1]# 만약 data['Summary'][1]값이 출력 안될때 

In [21]:
#print(a)